In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
import urllib.request, requests, json, re, time, os, difflib, itertools
import pandas as pd
from multiprocessing.dummy import Pool
from datetime import datetime
try:
    import httplib
except:
    import http.client as httplib
    
login = 'YOUR LOGIN'
password = 'YOUR PASSWORD'

url = 'https://ycharts.com/login?next=/companies/AAPL/market_cap'

In [2]:
csv_path = "market_capitalization_data"+os.sep+"csv"+os.sep
if not os.path.isdir(csv_path):
    os.makedirs(csv_path)

def check_internet():
    conn = httplib.HTTPConnection("www.google.com", timeout=5)
    try:
        conn.request("HEAD", "/")
        conn.close()
        return True
    except:
        conn.close()
        return False

In [3]:
ticker_file_path = "Yahoo Ticker Symbols - September 2017.xlsx"
temp_df = pd.read_excel(ticker_file_path)
temp_df = temp_df.drop(temp_df.columns[[5, 6, 7]], axis=1)
headers = temp_df.iloc[2]
df  = pd.DataFrame(temp_df.values[3:], columns=headers)
print("Total stocks:",len(df))

query_urls=[]
for ticker in df['Ticker']:
    query_urls.append("https://ycharts.com/login?next=/companies/"+ticker+"/market_cap")
    
df.head(3)

Total stocks: 106328


2,Ticker,Name,Exchange,Category Name,Country
0,OEDV,"Osage Exploration and Development, Inc.",PNK,NaN,USA
1,AAPL,Apple Inc.,NMS,Electronic Equipment,USA
2,BAC,Bank of America Corporation,NYQ,Money Center Banks,USA


In [4]:
def connection(url):
    driver = webdriver.Firefox(executable_path=r'C:\dr\geckodriver.exe')
    driver.get(url)

    for i in driver.get_cookies():
        if i['name'] == 'csrftoken':
            csrftoken = i['value']
        elif i['name'] == 'csrf':
            csrftoken = i['value']
    driver.get(url)

    s_username = driver.find_element_by_name('username')
    s_password = driver.find_element_by_name('password')
    s_continue = driver.find_element_by_css_selector('.btn-block')

    s_username.send_keys(login)
    s_password.send_keys(password)
    s_continue.click()
    time.sleep(2)
    driver.find_element_by_link_text('Quickflows').click()
    return driver

def fill_data(soup, data):
    div_r = soup.find('div',  attrs={"class" : "dataColRt"})
    div_l = soup.find('div',  attrs={"class" : "dataColLeft"})
    r_rows = div_r.find_all('tr')
    l_rows = div_l.find_all('tr')
    for rows_set in (l_rows, r_rows):
        for row in rows_set:
            row_data = []
            if row.find_all('td'):
                for cell in row.find_all('td'):
                    val = cell.get_text().strip()
                    row_data.append(val)
                data.append(row_data)
def data_scraping(url):
    data = []
    driver = connection(url)
    soup = BeautifulSoup(driver.page_source,"lxml")
    fill_data(soup, data) #first page
    
    while True: #others page
        try:
            driver.find_element_by_link_text('Next').click()
            time.sleep(0.1)
            soup = BeautifulSoup(driver.page_source,"lxml")
            for div in soup.find_all("div", {'class':'ng-hide'}): 
                div.decompose()
            fill_data(soup, data)
        except: break
    driver.close()
    return data

In [5]:
def get_market_cap_data(query_url,json_path,csv_path):
    
    stock_id=query_url.split('/market_cap')[0].split("companies/")[1]

    if os.path.exists(csv_path+stock_id+'.csv') and os.stat(csv_path+stock_id+'.csv').st_size != 0:
        print("<<<  Market capialization data of "+stock_id+" already exists")
        return
    
    while not check_internet():
        print("Could not connect, trying again in 5 seconds...")
        time.sleep(5)

    try:
        html = requests.get("https://ycharts.com/companies/"+stock_id+"/market_cap")
        if html.status_code == 200:
            data = data_scraping(query_url)

    except:
        print("|||  Market capialization data of "+stock_id+" doesn't exist")
        return

    else:
        try:
            for i, row in enumerate(data):
           #     data[i][0] = datetime.strptime(row[0], "%B %d, %Y")
                if row[1][-1] == "T":
                    data[i][1] =  int(float(row[1][:-1]) * 1000000000000)
                elif row[1][-1] == 'B':
                    data[i][1] =  int(float(row[1][:-1]) * 1000000000)
                elif row[1][-1] == 'M':
                    data[i][1] =  int(float(row[1][:-1]) * 1000000)
                else:
                    pass
                
            df = pd.DataFrame(data={"Date": pd.Series((v[0] for v in data)), "Market_cap": pd.Series((v[1] for v in data))})

            if os.path.exists(csv_path+stock_id+'.csv'):
                os.remove(csv_path+stock_id+'.csv')
            df.to_csv(csv_path+stock_id+'.csv', sep=',', index=None)
            
            print(">>>  Market capialization data of "+stock_id+" saved")
            
        except:
            print(">>>  Market capialization data of "+stock_id+" could not be saved")
        
        return

In [ ]:
with Pool(processes=3) as pool:
    pool.starmap(get_market_cap_data, zip(query_urls, itertools.repeat(json_path), itertools.repeat(csv_path)))
print("<|>  Market capialization data of all stocks saved")

>>>  Market capialization data of OEDV could not be saved
<<<  Market capialization data of AAPL already exists
<<<  Market capialization data of BAC already exists
<<<  Market capialization data of AMZN already exists
>>>  Market capialization data of JUP.L could not be saved>>>  Market capialization data of ORRP could not be saved

<<<  Market capialization data of T already exists
<<<  Market capialization data of GOOG already exists
<<<  Market capialization data of ORRAF already exists
<<<  Market capialization data of MO already exists
<<<  Market capialization data of DAL already exists
<<<  Market capialization data of AA already exists
<<<  Market capialization data of AXP already exists
<<<  Market capialization data of DD already exists
<<<  Market capialization data of BABA already exists
<<<  Market capialization data of ABT already exists
<<<  Market capialization data of UA already exists
<<<  Market capialization data of AMAT already exists
<<<  Market capialization dat